# Subset barcode runs
This Python Jupyter notebook is creates a copy of the barcode runs file that contains just specific samples.

First, import Python modules:

In [1]:
import os

import pandas as pd

import yaml

Read in the "master" barcode runs CSV file and the runs used to compute the escape scores for each sample:

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)

barcode_runs_file = os.path.join('../', config['barcode_runs'])
print(f"Reading barcode runs from {barcode_runs_file}")
barcode_runs = pd.read_csv(barcode_runs_file)

escape_score_samples_file = os.path.join('../', config['escape_score_samples'])
print(f"Reading escape score samples from {escape_score_samples_file}")
escape_score_samples = pd.read_csv(escape_score_samples_file)

Reading barcode runs from ../data/barcode_runs.csv
Reading escape score samples from ../results/escape_scores/samples.csv


Now read in the samples to subset:

In [3]:
samples_to_subset_df = pd.read_csv('samples_to_subset.csv')
print('Here are the samples we will subset to:')
display(samples_to_subset_df)

samples_to_subset = samples_to_subset_df['sample'].tolist()
assert len(samples_to_subset) == len(set(samples_to_subset)), 'duplicate samples to subset'

Here are the samples we will subset to:


,sample
0,2B04_400
1,2C02_400
2,2C03_400
3,mAb-2E06_400


Now get all the escape-score samples of interest:

In [4]:
if not set(samples_to_subset).issubset(escape_score_samples['name']):
    raise ValueError(f"Not all samples to subset are in the escape score samples.")
    
samples_subset = (
    escape_score_samples
    .query('name in @samples_to_subset')
    .reset_index(drop=True)
    )

print(f"Here are the samples for which we are subsetting barcode runs:")
display(samples_subset)

Here are the samples for which we are subsetting barcode runs:


,name,library,antibody,concentration,concentration_units,date,pre_sample,post_sample,frac_escape,pre_cells_sorted,post_cells_sorted
0,2B04_400,lib1,2B04,400,ng_per_mL,200721,expt_17-19-none-0-reference,expt_17-2B04-400-escape,0.207,2190044.0,160000000.0
1,2B04_400,lib2,2B04,400,ng_per_mL,200721,expt_17-19-none-0-reference,expt_17-2B04-400-escape,0.176,1991610.0,160000000.0
2,2C02_400,lib1,2C02,400,ng_per_mL,200615,expt_3-6-none-0-reference,expt_6-2C02-400-escape,0.157,1771544.0,160000000.0
3,2C02_400,lib2,2C02,400,ng_per_mL,200615,expt_3-6-none-0-reference,expt_6-2C02-400-escape,0.178,2163094.0,160000000.0
4,2C03_400,lib1,2C03,400,ng_per_mL,200721,expt_17-19-none-0-reference,expt_19-2C03-400-escape,0.219,2231282.0,160000000.0
5,2C03_400,lib2,2C03,400,ng_per_mL,200721,expt_17-19-none-0-reference,expt_19-2C03-400-escape,0.301,2756966.0,160000000.0
6,mAb-2E06_400,lib1,mAb-2E06,400,ng_per_mL,200810,expt_22-23-none-0-reference,expt_22-mAb-2E06-400-escape,0.043,434712.0,160000000.0
7,mAb-2E06_400,lib2,mAb-2E06,400,ng_per_mL,200810,expt_22-23-none-0-reference,expt_22-mAb-2E06-400-escape,0.044,491741.0,160000000.0


Now unfold the samples of interest into the actual relevant barcode runs (this requires getting both the pre- and post-selection run for each sample):

In [5]:
barcode_runs_subset = (
    samples_subset
    .melt(id_vars=['name', 'library'],
          value_vars=['pre_sample', 'post_sample'],
          value_name='sample',
          var_name='sample_type')
    [['library', 'sample']]
    .drop_duplicates()
    .reset_index(drop=True)
    .merge(barcode_runs,
           how='left',
           on=['library', 'sample'],
           validate='one_to_one',
           )
    )

assert barcode_runs_subset['R1'].notnull().all(), 'some barcode runs missing R1'

barcode_runs_subset_file = 'barcode_runs_subset.csv'
print(f"Here are the subsetted barcode runs. Writing to {barcode_runs_subset_file}")
barcode_runs_subset.to_csv(barcode_runs_subset_file, index=False)
display(barcode_runs_subset)

Here are the subsetted barcode runs. Writing to barcode_runs_subset.csv


,library,sample,date,experiment,antibody,concentration,concentration_units,group,selection,frac_escape,cells_sorted,R1
0,lib1,expt_17-19-none-0-reference,200721,expt_17-19,none,0,ng_per_mL,none,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200727_D00300_10...
1,lib2,expt_17-19-none-0-reference,200721,expt_17-19,none,0,ng_per_mL,none,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200727_D00300_10...
2,lib1,expt_3-6-none-0-reference,200615,expt_3-6,none,0,ng_per_mL,none,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200618_D00300_09...
3,lib2,expt_3-6-none-0-reference,200615,expt_3-6,none,0,ng_per_mL,none,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200618_D00300_09...
4,lib1,expt_22-23-none-0-reference,200810,expt_22-23,none,0,ng_per_mL,none,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200812_D00300_10...
5,lib2,expt_22-23-none-0-reference,200810,expt_22-23,none,0,ng_per_mL,none,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200812_D00300_10...
6,lib1,expt_17-2B04-400-escape,200721,expt_17,2B04,400,ng_per_mL,Ellebedy,escape,0.207,2190044.0,/shared/ngs/illumina/agreaney/200727_D00300_10...
7,lib2,expt_17-2B04-400-escape,200721,expt_17,2B04,400,ng_per_mL,Ellebedy,escape,0.176,1991610.0,/shared/ngs/illumina/agreaney/200727_D00300_10...
8,lib1,expt_6-2C02-400-escape,200615,expt_6,2C02,400,ng_per_mL,Ellebedy,escape,0.157,1771544.0,/shared/ngs/illumina/agreaney/200618_D00300_09...
9,lib2,expt_6-2C02-400-escape,200615,expt_6,2C02,400,ng_per_mL,Ellebedy,escape,0.178,2163094.0,/shared/ngs/illumina/agreaney/200618_D00300_09...
